In [ ]:
# =========================
# IMPORT LIBRARIES
# =========================
import zipfile, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# =========================
# SETUP PATHS AND UNZIP DATA
# =========================
base_path = "/kaggle/input/facial-keypoints-detection"
work_path = "/kaggle/working"

# Unzip training.zip
with zipfile.ZipFile(f"{base_path}/training.zip", 'r') as zip_ref:
    zip_ref.extractall(work_path)

# Unzip test.zip
with zipfile.ZipFile(f"{base_path}/test.zip", 'r') as zip_ref:
    zip_ref.extractall(work_path)

print("Extracted files:", os.listdir(work_path))

# =========================
# LOAD CSV FILES
# =========================
TRAIN_CSV = f"{work_path}/training.csv"
TEST_CSV  = f"{work_path}/test.csv"
LOOKUP_CSV = f"{base_path}/IdLookupTable.csv"

train = pd.read_csv(TRAIN_CSV)
test  = pd.read_csv(TEST_CSV)
lookup = pd.read_csv(LOOKUP_CSV)

print("TRAIN shape:", train.shape)
print("TEST shape:", test.shape)
print("LOOKUP shape:", lookup.shape)

# =========================
# HELPER FUNCTIONS
# =========================
def image_from_str(img_str):
    arr = np.fromstring(img_str, sep=' ')
    return arr.reshape(96, 96)

def prepare_images(df):
    imgs = np.stack(df['Image'].apply(lambda s: image_from_str(s)).values)
    imgs = imgs.astype('float32') / 255.0
    imgs = imgs.reshape(-1, 96, 96, 1)
    return imgs

# =========================
# VISUALIZE SAMPLE IMAGE
# =========================
sample = train.dropna().iloc[0]
img = image_from_str(sample['Image'])
plt.imshow(img, cmap='gray')
kp = sample.drop('Image').values.astype(float)
plt.scatter(kp[0::2], kp[1::2], c='r', s=10)
plt.title("Sample image with keypoints")
plt.show()

# =========================
# CLEAN AND PREPARE DATA
# =========================
train_clean = train.dropna().reset_index(drop=True)
print("After dropna:", train_clean.shape)

X = prepare_images(train_clean)
y = train_clean.drop(columns=['Image']).values.astype('float32')
kp_columns = train_clean.drop(columns=['Image']).columns.tolist()
print("Keypoint columns:", kp_columns)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.15, random_state=SEED
)
print("X_train shape:", X_train.shape, "X_val shape:", X_val.shape)

# =========================
# BUILD CNN MODEL
# =========================
def build_model():
    model = Sequential([
        Input(shape=(96, 96, 1)),
        Conv2D(32, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dense(len(kp_columns))  # 30 keypoints
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

model = build_model()
model.summary()

# =========================
# TRAIN MODEL
# =========================
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=64,
    callbacks=[es, mc],
    verbose=2
)

# Plot training / validation loss
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training / Validation Loss')
plt.show()

# =========================
# PREPARE TEST DATA AND PREDICT
# =========================
test_images = prepare_images(test)
model.load_weights('best_model.h5')
preds = model.predict(test_images)
print("Predictions shape:", preds.shape)  # should be (n_test, 30)

# =========================
# CREATE CORRECT SUBMISSION (CLIPPED TO [0,96])
# =========================
submission = lookup.copy()

submission['Location'] = submission.apply(
    lambda r: np.clip(
        preds[int(r['ImageId']) - 1, kp_columns.index(r['FeatureName'])],
        0, 96  # clip to valid image coordinates
    ),
    axis=1
)

# Keep only RowId and Location columns
submission = submission[['RowId', 'Location']]
submission.to_csv('submission.csv', index=False)
print("✅ submission.csv ready for Kaggle (coordinates clipped to [0,96])!")
